In [1]:
%lsmagics
%endpoint http://54.36.123.165:8890/sparql/

Available magics:
%auth  %display  %endpoint  %format  %graph  %header  %http_header  %lang  %load  %log  %lsmagics  %method  %outfile  %prefix  %qparam  %show

%auth (basic|digest|none) <username> <passwd> : send HTTP authentication (use env:<var> to get values from environment variables)
%display raw | table [withtypes] | diagram [svg|png] [withliterals] : set display format
%endpoint <url> : set SPARQL endpoint. **REQUIRED**
%format JSON | N3 | XML | default | any | none : set requested result format
%graph <uri> : set default graph for the queries
%header <string> | OFF : add a persistent SPARQL header line before all queries, or delete all defined headers
%http_header <name> [<value>] : add (or delete) an arbitrary HTTP header to all queries
%lang <lang> [...] | default | all : language(s) preferred for labels
%load <filename> : load a file with magic lines and process them
%log critical | error | warning | info | debug : set logging level
%lsmagics  : list all magics
%method get | post : set HTTP method
%outfile <filename> | off : save raw output to a file (use "%d" in name to add cell number, "off" to cancel saving)
%prefix <name> [<uri>] : set (or delete) a persistent URI prefix for all queries
%qparam <name> [<value>] : add (or delete) a persistent custom parameter to all queries
%show <n> | all : maximum number of shown results
Endpoint set to: http://54.36.123.165:8890/sparql/

In [2]:
%show all
# Request whatever format is appropriate for the query type
%format default

# Activate table output
# %display table
%display table

Result maximum size: unlimited
Request format: DEFAULT
Display: table

In [3]:
%prefix rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
%prefix rdfs:    <http://www.w3.org/2000/01/rdf-schema#> 
%prefix oa:     <http://www.w3.org/ns/oa#>
%prefix skos: <http://www.w3.org/2004/02/skos/core#>
%prefix schema:  <http://schema.org/>
%prefix paragraph: <http://www.zoomathia.com/>
%prefix ex: <http://ex_building/>

Prefix set: rdf: = <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
Prefix set: rdfs: = <http://www.w3.org/2000/01/rdf-schema#>
Prefix set: oa: = <http://www.w3.org/ns/oa#>
Prefix set: skos: = <http://www.w3.org/2004/02/skos/core#>
Prefix set: schema: = <http://schema.org/>
Prefix set: paragraph: = <http://www.zoomathia.com/>
Prefix set: ex: = <http://ex_building/>

# CQ1 - Quels sont les animaux qui construisent un habitat ?

Cette question exprime le besoin du chercheur à cibler les passages dans le texte où l'auteur mentionne un animal et la construction d'un habitat.

   <u>__Reformulation__</u> : La reformulation de la requête SPARQL exprime cette requête comme étant la recherche: (1) d'une annotation Annotation ayant pour body le concept d'un animal et pour cible le paragraphe où cette mention est faite, (2) d'une Annotation ayant pour body le concept de construction d'habitat (enfant du concept "house building") et pour cible le paragraphe où cette mention est faite.

   <u>__Restriction__</u> : Pour restreindre les résultats et apporté de la cohérence entre les deux concepts, l'apparition des deux concepts mentionnés plus haut doivent faire partie du même paragraphe.

   <u>__Note__</u> : Deux collections rassemblent les concepts d'animaux: (1) la collection Zoonyme (Zoonyms) et (2) la collection Archéotaxon (Ancient class). Ces deux collections partagent une majeure partie des concepts animal. Les deux collections sont inclues pour maximiser le nombre de concept trouvé.


In [4]:
SELECT DISTINCT ?chapter_id ?paragraph_id ?text ?label_animal ?mention_habitat ?label_habitat 
WHERE {
     ?animal_annotation a oa:Annotation; oa:hasBody ?animal_concept;
        oa:hasTarget [
          oa:hasSource ?paragraph;
          oa:hasSelector [ oa:exact ?mention_animal;]].
    ?habitat_annotation a oa:Annotation; oa:hasBody ?habitat_concept;
        oa:hasTarget [
          oa:hasSource ?paragraph;
          oa:hasSelector [ oa:exact ?mention_habitat; ]].

    ?animal_concept a skos:Concept; skos:prefLabel ?label_animal.
    ?animal_collection a skos:Collection; skos:member ?animal_concept;
        skos:prefLabel ?label.
    
    ?habitat_concept a skos:Concept; skos:prefLabel ?label_habitat;
        skos:broader ?habitat_generic_concept.
    ?habitat_generic_concept a skos:Concept; skos:prefLabel "house building"@en.
    
    ?paragraph schema:text ?text;
               schema:identifier ?paragraph_id;
               schema:isPartOf ?chapter.
    
    ?chapter schema:identifier ?chapter_id.
    
    filter(lang(?label_animal) = "en")
    filter(lang(?label_habitat) = "en")
    filter(?label in ("Ancient class"@en, "Zoonyms"@en))
}
ORDER BY ?paragraph

chapter_id,paragraph_id,text,label_animal,mention_habitat,label_habitat
10,10,"haliaetus tantum inplumes etiamnum pullos suos percutiens subinde cogit adversos intueri solis radios et, si coniventem umectantemque animadvertit, praecipitat e nido velut adulterinum atque degenerem. illum, cuius acies firma contra stetit, educat.",osprey,"haliaetus tantum inplumes etiamnum pullos suos percutiens subinde cogit adversos intueri solis radios et, si coniventem umectantemque animadvertit, praecipitat e nido velut adulterinum atque degenerem. illum, cuius acies firma contra stetit, educat.",nest
10,10,"haliaetus tantum inplumes etiamnum pullos suos percutiens subinde cogit adversos intueri solis radios et, si coniventem umectantemque animadvertit, praecipitat e nido velut adulterinum atque degenerem. illum, cuius acies firma contra stetit, educat.",eagle,"haliaetus tantum inplumes etiamnum pullos suos percutiens subinde cogit adversos intueri solis radios et, si coniventem umectantemque animadvertit, praecipitat e nido velut adulterinum atque degenerem. illum, cuius acies firma contra stetit, educat.",nest
10,100,"Perdices spina et frutice sic muniunt receptaculum, ut contra feram abunde vallentur. ovis stragulum molle pulvere contumulant nec in quo loco peperere incubant: ne cui frequentior conversatio suspecta sit, transferunt alio. illae quidem et maritos suos fallunt, quoniam intemperantia libidinis frangunt earum ova, ne incubando detineantur. tunc inter se dimicant mares desiderio feminarum; victum aiunt venerem pati.",WILD BEAST,"Perdices spina et frutice sic muniunt receptaculum, ut contra feram abunde vallentur. ovis stragulum molle pulvere contumulant nec in quo loco peperere incubant: ne cui frequentior conversatio suspecta sit, transferunt alio. illae quidem et maritos suos fallunt, quoniam intemperantia libidinis frangunt earum ova, ne incubando detineantur. tunc inter se dimicant mares desiderio feminarum; victum aiunt venerem pati.",nest
10,100,"Perdices spina et frutice sic muniunt receptaculum, ut contra feram abunde vallentur. ovis stragulum molle pulvere contumulant nec in quo loco peperere incubant: ne cui frequentior conversatio suspecta sit, transferunt alio. illae quidem et maritos suos fallunt, quoniam intemperantia libidinis frangunt earum ova, ne incubando detineantur. tunc inter se dimicant mares desiderio feminarum; victum aiunt venerem pati.",partridge,"Perdices spina et frutice sic muniunt receptaculum, ut contra feram abunde vallentur. ovis stragulum molle pulvere contumulant nec in quo loco peperere incubant: ne cui frequentior conversatio suspecta sit, transferunt alio. illae quidem et maritos suos fallunt, quoniam intemperantia libidinis frangunt earum ova, ne incubando detineantur. tunc inter se dimicant mares desiderio feminarum; victum aiunt venerem pati.",nest
10,103,"si ad nidum is coepit accedere, procurrit ad pedes eius feta, praegravem aut delumbem sese simulans, subitoque in procursu aut brevi aliquo volatu cadit, ut fracta ala aut pedibus, at procurrit iterum iamiam prehensurum effugiens spemque frustrans, donec in diversum abducat a nidis. eadem in pavore, libera ac materna vacans cura, in sulco resupina glaeba se terrae pedibus adprehensa operit. perdicum vita et ad sedecim annos durare existimatur.",partridge,"si ad nidum is coepit accedere, procurrit ad pedes eius feta, praegravem aut delumbem sese simulans, subitoque in procursu aut brevi aliquo volatu cadit, ut fracta ala aut pedibus, at procurrit iterum iamiam prehensurum effugiens spemque frustrans, donec in diversum abducat a nidis. eadem in pavore, libera ac materna vacans cura, in sulco resupina glaeba se terrae pedibus adprehensa operit. perdicum vita et ad sedecim annos durare existimatur.",nest
10,104,"Ab iis columbarum maxime spectantur simili ratione mores. inest pudicitia illis plurima et neutri nota adulteria. coniugi fidem non violant communemque servant domum. nisi caelebs aut vidua nidum non relinquit. et imperiosos mares

# CQ2 - Quelles anecdotes mettent en relation un homme et un animal ?

Cette question exprime le besoin du chercheur à cibler les passages dans le texte où l'auteur mentionne une relation entre un animal et l'homme. Cette relation est décrite au travers d'une anecdote. Les relations comme la chasse et autres ne sont pas attendu. Uniquement les situations individuelles.

   <u>__Reformulation__</u> : La reformulation de la requête SPARQL est très complexe. Pour étudier les possibilités offertes, la requête sera décomposé en plusieurs requêtes portant sur différent concept.

   <u>__Restriction__</u> : Pour restreindre les résultats et apporté de la cohérence entre les deux concepts, l'apparition des concepts mentionnés plus haut doivent faire partie du même paragraphe.

   <u>__Note__</u> : Une anecdote est un concept très difficile à caractériser. La définition proposée est la suivante: une anecdote est un évènemment unique, daté mettant en relation un antroponyme et un animal dans un lieu précisé. Toutes ces informations peuvent ne pas apparaitre dans le texte. De plus, il est possible que lors de l'annotation, le concept d'anecdote n'ait pas été renseigné.


## CQ2.1 - Recherche restreinte : Animal et Anecdote
Cette première approche permet de voir le nombre d'anecdote présent dans les textes.

In [5]:
SELECT DISTINCT ?chapter_id ?paragraph_id ?name_animal ?mention_animal ?mention_anecdote WHERE {
    
?annotation1 a oa:Annotation;
    oa:hasBody ?animal;
    oa:hasTarget [
        oa:hasSource ?paragraph;
        oa:hasSelector [
            oa:exact ?mention_animal; oa:start ?start; oa:end ?end
        ]
    ].

?annotation2 a oa:Annotation;
    oa:hasBody ?anecdote;
    oa:hasTarget [
        oa:hasSource ?paragraph;
        oa:hasSelector [
            oa:exact ?mention_anecdote; oa:start ?start; oa:end ?end
        ]
    ].

?animal a skos:Concept;
    skos:prefLabel ?name_animal.
    
?animal_collection a skos:Collection;
    skos:prefLabel "Ancient class"@en;
    skos:member ?animal.

?anecdote skos:prefLabel "anecdote"@en.

?paragraph schema:text ?text;
           schema:identifier ?paragraph_id;
           schema:isPartOf ?chapter.

?chapter schema:identifier ?chapter_id.

FILTER (lang(?name_animal) = "en").
}
ORDER BY ?paragraph

chapter_id,paragraph_id,name_animal,mention_animal,mention_anecdote
10,121,raven,"Reddatur et corvis sua gratia, indignatione quoque populi Romani testata, non solum conscientia. Tiberio principe ex fetu supra Castorum aedem genito pullus in adpositam sutrinam devolavit, etiam religione commendatus officinae domino. is mature sermoni adsuefactus, omnibus matutinis evolans in rostra in forum versus, Tiberium, dein Germanicum et Drusum Caesares nominatim, mox transeuntem populum Romanum salutabat, postea ad tabernam remeans, plurium annorum adsiduo officio mirus.","Reddatur et corvis sua gratia, indignatione quoque populi Romani testata, non solum conscientia. Tiberio principe ex fetu supra Castorum aedem genito pullus in adpositam sutrinam devolavit, etiam religione commendatus officinae domino. is mature sermoni adsuefactus, omnibus matutinis evolans in rostra in forum versus, Tiberium, dein Germanicum et Drusum Caesares nominatim, mox transeuntem populum Romanum salutabat, postea ad tabernam remeans, plurium annorum adsiduo officio mirus."
10,122,raven,"hunc sive aemulatione vicinitatis manceps proximae sutrinae sive iracundia subita, ut voluit videri, excrementis eius posita calceis macula, exanimavit, tanta plebei consternatione, ut primo pulsus ex ea regione, mox interemptus sit funusque aliti innumeris celebratum exequiis, constratum lectum super Aethiopum duorum umeros, praecedente tibicine et coronis omnium generum ad rogum usque, qui constructus dextra viae Appiae ad secundum lapidem in campo Rediculi appellato fuit.","hunc sive aemulatione vicinitatis manceps proximae sutrinae sive iracundia subita, ut voluit videri, excrementis eius posita calceis macula, exanimavit, tanta plebei consternatione, ut primo pulsus ex ea regione, mox interemptus sit funusque aliti innumeris celebratum exequiis, constratum lectum super Aethiopum duorum umeros, praecedente tibicine et coronis omnium generum ad rogum usque, qui constructus dextra viae Appiae ad secundum lapidem in campo Rediculi appellato fuit."
10,123,raven,"adeo satis iusta causa populo Romano visa est exequiarum ingenium avis aut supplicii de cive Romano in ea urbe, in qua multorum principum nemo deduxerat funus, Scipionis vero Aemiliani post Carthaginem Numantiamque deletas ab eo nemo vindicaverat mortem.","adeo satis iusta causa populo Romano visa est exequiarum ingenium avis aut supplicii de cive Romano in ea urbe, in qua multorum principum nemo deduxerat funus, Scipionis vero Aemiliani post Carthaginem Numantiamque deletas ab eo nemo vindicaverat mortem."
10,124,raven,"hoc gestum M. Servilio C. Cestio cos. a. d. V kal. Apriles. nunc quoque erat in urbe Roma, haec prodente me, equitis Romani cornix e Baetica primum colore mira admodum nigro, dein plura contexta verba exprimens et alia atque alia crebro addiscens. nec non et recens fama Crateri Monocerotis cognomine in Erizena regione Asiae corvorum opera venantis eo quod devehebat in silvas eos insidentes corniculo umerisque; illi vestigabant agebantque eo perducta consuetudine, ut exeuntem sic comitarentur et feri.","hoc gestum M. Servilio C. Cestio cos. a. d. V kal. Apriles. nunc quoque erat in urbe Roma, haec prodente me, equitis Romani cornix e Baetica primum colore mira admodum nigro, dein plura contexta verba exprimens et alia atque alia crebro addiscens. nec non et recens fama Crateri Monocerotis cognomine in Erizena regione Asiae corvorum opera venantis eo quod devehebat in silvas eos insidentes corniculo umerisque; illi vestigabant agebantque eo perducta consuetudine, ut exeuntem sic comitarentur et feri."
10,124,crow,"hoc gestum M. Servilio C. Cestio cos. a. d. V kal. Apriles. nunc quoque erat in urbe Roma, haec prodente me, equitis Romani cornix e Baetica primum colore mira admodum nigro, dein plura contexta verba exprimens et alia atque alia crebro addiscens. nec non et recens fama Crateri Monocerotis cognomine in Erizena regione Asiae corvorum opera venantis eo quod devehebat in silvas eos insidentes corn

In [6]:
SELECT DISTINCT ?chapter_id ?paragraph_id ?name_animal ?mention_animal ?mention_anecdote WHERE {
    
?annotation1 a oa:Annotation;
    oa:hasBody ?animal;
    oa:hasTarget [
        oa:hasSource ?paragraph;
        oa:hasSelector [
            oa:exact ?mention_animal; oa:start ?start; oa:end ?end
        ]
    ].

?annotation2 a oa:Annotation;
    oa:hasBody ?anecdote;
    oa:hasTarget [
        oa:hasSource ?paragraph;
        oa:hasSelector [
            oa:exact ?mention_anecdote; oa:start ?start; oa:end ?end
        ]
    ].

?annotation3 a oa:Annotation;
    oa:hasBody ?relationship;
    oa:hasTarget [
        oa:hasSource ?paragraph;
        oa:hasSelector [
            oa:exact ?mention_relationship; oa:start ?start; oa:end ?end
        ]
    ].

?animal a skos:Concept;
    skos:prefLabel ?name_animal.
    
?animal_collection a skos:Collection;
    skos:prefLabel "Ancient class"@en;
    skos:member ?animal.

?anecdote skos:prefLabel "anecdote"@en.
    
?relationship skos:prefLabel ?name_relationship;
    skos:broader+ ?generic_relationship.
?generic_relationship skos:prefLabel "special relationship"@en.
    
?paragraph schema:text ?text;
           schema:identifier ?paragraph_id;
           schema:isPartOf ?chapter.

?chapter schema:identifier ?chapter_id.

FILTER (lang(?name_animal) = "en").
}
ORDER BY ?paragraph

chapter_id,paragraph_id,name_animal,mention_animal,mention_anecdote


<hr>

# QC3 - Quels sont les oiseaux qui sont consommés (gastronomie)

This SPARQL query allows researchers to identify the animals that have been used for gastronomic use

In [17]:
SELECT DISTINCT ?paragraph ?name_animal ?mention_animal ?name_conso ?mention_conso WHERE {
  ?annotation1 oa:hasBody ?animal; oa:hasTarget [
    oa:hasSource ?paragraph;
    oa:hasSelector [
        oa:exact ?mention_animal
        ]
    ].

  ?animal skos:prefLabel ?name_animal; skos:broader+ ?animal_generic.
    
  ?animal_generic skos:prefLabel ?name_animal_generic.

  ?annotation2 oa:hasBody ?conso;
        oa:hasTarget  [oa:hasSource ?paragraph;
      oa:hasSelector [oa:exact ?mention_conso]].

  ?conso skos:prefLabel ?name_conso; skos:broader+ ?conso_generique.
  ?conso_generique skos:prefLabel "animal in human nourishing"@en.
FILTER (str(?name_animal_generic) in ("BIRD","BIRD WITH CLAW","BIRD WITH TOES","WATERBIRD","NIGHT BIRD","MIGRATORY BIRD", "PALMIPED BIRD")).
}
ORDER BY ?paragraph

paragraph,name_animal,mention_animal,name_conso,mention_conso


# CQ4

This SPARQL query allows researchers to identify the animals that have been used for medical purposes and more precisely the part of the body of the animal.

In [8]:
SELECT DISTINCT ?paragraph ?name_animal ?mention1 ?mention2 WHERE {
  ?annotation1 a oa:Annotation;
              oa:hasBody ?animal;
              oa:hasTarget ?target1.
  ?target1 oa:hasSource ?paragraph;
     oa:hasSelector ?selector1.

?annotation2 a oa:Annotation;
              oa:hasBody ?use;
              oa:hasTarget ?target2.
  ?target2 oa:hasSource ?paragraph;
     oa:hasSelector ?selector2.
    
  ?selector1 oa:exact ?mention1.
  ?selector2 oa:exact ?mention2.

  ?animal a skos:Concept;
       skos:prefLabel ?name_animal.
    
  ?animal_collection a skos:Collection;
       skos:prefLabel "Ancient class"@en;
       skos:member ?animal.

  ?use skos:prefLabel ?name_use;
       skos:broader+ ?use_generique.
  ?use_generique skos:prefLabel "medical use of animal"@en.
  
  FILTER (lang(?name_animal) = "en").
  FILTER (lang(?name_use) = "en")
}
ORDER BY ?paragraph

paragraph,name_animal,mention1,mention2
http://www.zoomathia.com/Pline/10/55,goose,"Aliud repperit Syriae pars quae Commagene vocatur, adipem eorum in vase aereo cum cinnamo nive multa obrutum ac rigore gelido maceratum ad usum praeclari medicaminis, quod ab gente dicitur Commagenum.","Aliud repperit Syriae pars quae Commagene vocatur, adipem eorum in vase aereo cum cinnamo nive multa obrutum ac rigore gelido maceratum ad usum praeclari medicaminis, quod ab gente dicitur Commagenum."
http://www.zoomathia.com/Pline/11/118,gnat,"spirant. si adcrevere, faciunt chrysallidem. ficarios culices caprificus generat, cantharidas vermiculi ficorum et piri et peuces et cynacanthae et rosae. venenum hoc remedia secum habet: alae medentur, quibus demptis letale est. rursus alia genera culicum acescens natura gignit, quippe cum et in nive inveniantur vetustiore vermiculi rutili_nam et ipsa nix vetustate rufescit_ hirti pilis, in Media quidem candidi, grandiores torpentesque.","spirant. si adcrevere, faciunt chrysallidem. ficarios culices caprificus generat, cantharidas vermiculi ficorum et piri et peuces et cynacanthae et rosae. venenum hoc remedia secum habet: alae medentur, quibus demptis letale est. rursus alia genera culicum acescens natura gignit, quippe cum et in nive inveniantur vetustiore vermiculi rutili_nam et ipsa nix vetustate rufescit_ hirti pilis, in Media quidem candidi, grandiores torpentesque."
http://www.zoomathia.com/Pline/11/118,VERMIS,"spirant. si adcrevere, faciunt chrysallidem. ficarios culices caprificus generat, cantharidas vermiculi ficorum et piri et peuces et cynacanthae et rosae. venenum hoc remedia secum habet: alae medentur, quibus demptis letale est. rursus alia genera culicum acescens natura gignit, quippe cum et in nive inveniantur vetustiore vermiculi rutili_nam et ipsa nix vetustate rufescit_ hirti pilis, in Media quidem candidi, grandiores torpentesque.","spirant. si adcrevere, faciunt chrysallidem. ficarios culices caprificus generat, cantharidas vermiculi ficorum et piri et peuces et cynacanthae et rosae. venenum hoc remedia secum habet: alae medentur, quibus demptis letale est. rursus alia genera culicum acescens natura gignit, quippe cum et in nive inveniantur vetustiore vermiculi rutili_nam et ipsa nix vetustate rufescit_ hirti pilis, in Media quidem candidi, grandiores torpentesque."
http://www.zoomathia.com/Pline/11/118,caterpillar with robe,"spirant. si adcrevere, faciunt chrysallidem. ficarios culices caprificus generat, cantharidas vermiculi ficorum et piri et peuces et cynacanthae et rosae. venenum hoc remedia secum habet: alae medentur, quibus demptis letale est. rursus alia genera culicum acescens natura gignit, quippe cum et in nive inveniantur vetustiore vermiculi rutili_nam et ipsa nix vetustate rufescit_ hirti pilis, in Media quidem candidi, grandiores torpentesque.","spirant. si adcrevere, faciunt chrysallidem. ficarios culices caprificus generat, cantharidas vermiculi ficorum et piri et peuces et cynacanthae et rosae. venenum hoc remedia secum habet: alae medentur, quibus demptis letale est. rursus alia genera culicum acescens natura gignit, quippe cum et in nive inveniantur vetustiore vermiculi rutili_nam et ipsa nix vetustate rufescit_ hirti pilis, in Media quidem candidi, grandiores torpentesque."
http://www.zoomathia.com/Pline/11/118,fig gnat,"spirant. si adcrevere, faciunt chrysallidem. ficarios culices caprificus generat, cantharidas vermiculi ficorum et piri et peuces et cynacanthae et rosae. venenum hoc remedia secum habet: alae medentur, quibus demptis letale est. rursus alia genera culicum acescens natura gignit, quippe cum et in nive inveniantur vetustiore vermiculi rutili_nam et ipsa nix vetustate rufescit_ hirti pilis, in Media quidem candidi, grandiores torpentesque.","spirant. si adcrevere, faciunt chrysallidem. ficarios culices caprificus generat, cantharidas vermiculi ficorum et piri et peuces et cynacanthae et rosae. venenum hoc remedia secum habet: alae medentur, quib

# CQ5

This SPARQL query allow to identify in the text the communication mode (language) between annimals.

In [9]:
SELECT DISTINCT ?paragraph ?name_animal ?mention_animal ?mention_social WHERE {
    
  ?annotation1 a oa:Annotation;
              oa:hasBody ?animal;
              oa:hasTarget ?target1.
  ?target1 oa:hasSource ?paragraph;
     oa:hasSelector ?selector.
    
  ?selector oa:exact ?mention_animal.

  ?animal a skos:Concept;
       skos:prefLabel ?name_animal.
    
  ?animal_collection a skos:Collection;
       skos:prefLabel "Ancient class"@en;
       skos:member ?animal.

 ?annotation2 oa:hasBody ?social;
        oa:hasTarget ?target2.
  ?target2 oa:hasSource ?paragraph;
      oa:hasSelector ?selector2.
  ?selector2 oa:exact ?mention_social.
    
  ?social skos:prefLabel "speech"@en.
      

    
  FILTER (lang(?name_animal) = "en").
}
ORDER BY ?paragraph

paragraph,name_animal,mention_animal,mention_social
http://www.zoomathia.com/Pline/10/117,parrot,"super omnia humanas voces reddunt, psittaci quidem etiam sermocinantes. India hanc avem mittit, siptacen vocat, viridem toto corpore, torque tantum miniato in cervice distinctam. imperatores salutat et quae accipit verba pronuntiat, in vino praecipue lasciva. capiti eius duritia eadem quae rostro. hoc, cum loqui discit, ferreo verberatur radio; non sentit aliter ictus. cum devolat, rostro se excipit, illi innititur levioremque ita se pedum infirmitati facit.","super omnia humanas voces reddunt, psittaci quidem etiam sermocinantes. India hanc avem mittit, siptacen vocat, viridem toto corpore, torque tantum miniato in cervice distinctam. imperatores salutat et quae accipit verba pronuntiat, in vino praecipue lasciva. capiti eius duritia eadem quae rostro. hoc, cum loqui discit, ferreo verberatur radio; non sentit aliter ictus. cum devolat, rostro se excipit, illi innititur levioremque ita se pedum infirmitati facit."
http://www.zoomathia.com/Pline/10/117,BIRD,"super omnia humanas voces reddunt, psittaci quidem etiam sermocinantes. India hanc avem mittit, siptacen vocat, viridem toto corpore, torque tantum miniato in cervice distinctam. imperatores salutat et quae accipit verba pronuntiat, in vino praecipue lasciva. capiti eius duritia eadem quae rostro. hoc, cum loqui discit, ferreo verberatur radio; non sentit aliter ictus. cum devolat, rostro se excipit, illi innititur levioremque ita se pedum infirmitati facit.","super omnia humanas voces reddunt, psittaci quidem etiam sermocinantes. India hanc avem mittit, siptacen vocat, viridem toto corpore, torque tantum miniato in cervice distinctam. imperatores salutat et quae accipit verba pronuntiat, in vino praecipue lasciva. capiti eius duritia eadem quae rostro. hoc, cum loqui discit, ferreo verberatur radio; non sentit aliter ictus. cum devolat, rostro se excipit, illi innititur levioremque ita se pedum infirmitati facit."
http://www.zoomathia.com/Pline/10/118,magpie,"Minor nobilitas, quia non ex longinquo venit, sed expressior loquacitas certo generi picarum est. adamant verba quae loquantur nec discunt, sed diligunt meditantesque intra semet curam atque cogitationem, intentionem non occultant. constat emori victas difficultate verbi ac, nisi subinde eadem audiant, memoria falli quaerentesque mirum in modum hilarari, si interim audierint id verbum. nec vulgaris his forma, quamvis non spectanda: satis illis decoris in specie sermonis humani est.","Minor nobilitas, quia non ex longinquo venit, sed expressior loquacitas certo generi picarum est. adamant verba quae loquantur nec discunt, sed diligunt meditantesque intra semet curam atque cogitationem, intentionem non occultant. constat emori victas difficultate verbi ac, nisi subinde eadem audiant, memoria falli quaerentesque mirum in modum hilarari, si interim audierint id verbum. nec vulgaris his forma, quamvis non spectanda: satis illis decoris in specie sermonis humani est."
http://www.zoomathia.com/Pline/10/120,starling,"quamquam id paene in omnibus contigit: Agrippina Claudii Caesaris turdum habuit, quod numquam ante, imitantem sermones hominum. cum haec proderem, habebant et Caesares iuvenes sturnum, item luscinias Graeco ac Latino sermone dociles, praeterea meditantes assidue et in diem nova loquentes, longiore etiam contextu. docentur secreto et ubi nulla alia vox misceatur, adsidente qui crebro dicat ea, quae condita velit, ac cibis blandiente.","quamquam id paene in omnibus contigit: Agrippina Claudii Caesaris turdum habuit, quod numquam ante, imitantem sermones hominum. cum haec proderem, habebant et Caesares iuvenes sturnum, item luscinias Graeco ac Latino sermone dociles, praeterea meditantes assidue et in diem nova loquentes, longiore etiam contextu. docentur secreto et ubi nulla alia vox misceatur, adsidente qui crebro dicat ea, quae condita velit, ac cibis blandiente."
http://www.zoomathia.com/Pline/10/120,nightingale,"q

# CQ 6

This query allows to identify the data provided about the gestation time of the animals

In [10]:
SELECT DISTINCT ?paragraph ?name_animal ?mention_animal ?mention_pregnancy WHERE {
?annotation1 a oa:Annotation;
              oa:hasBody ?animal;
              oa:hasTarget ?target1.
  ?target1 oa:hasSource ?paragraph;
     oa:hasSelector ?selector.
    
  ?selector oa:exact ?mention_animal.

  ?animal a skos:Concept;
       skos:prefLabel ?name_animal.
    
  ?animal_collection a skos:Collection;
       skos:prefLabel "Ancient class"@en;
       skos:member ?animal.

  ?annotation2 oa:hasBody ?pregnancy;
        oa:hasTarget ?target2.
  ?target2 oa:hasSource ?paragraph;
      oa:hasSelector ?selector2.
  ?selector2 oa:exact ?mention_pregnancy.

  ?pregnancy skos:prefLabel "length of pregnancy"@en.

  FILTER (lang(?name_animal) = "en").
}
ORDER BY ?paragraph

paragraph,name_animal,mention_animal,mention_pregnancy
http://www.zoomathia.com/Pline/10/147,swallow1,"columbae deciens anno pariunt, quaedam et undeciens, in Aegypto vero etiam brumali mense. hirundines et merulae et palumbi et turtures bis anno pariunt, ceterae aves fere semel. turdi, in cacuminibus arborum luto nidificantes paene contextim, in secessu generant. a coitu X diebus ova maturescunt in utero, vexatis autem gallinae et columbae pinna evulsa aliave simili iniuria diutius.","columbae deciens anno pariunt, quaedam et undeciens, in Aegypto vero etiam brumali mense. hirundines et merulae et palumbi et turtures bis anno pariunt, ceterae aves fere semel. turdi, in cacuminibus arborum luto nidificantes paene contextim, in secessu generant. a coitu X diebus ova maturescunt in utero, vexatis autem gallinae et columbae pinna evulsa aliave simili iniuria diutius."
http://www.zoomathia.com/Pline/10/147,ringdove,"columbae deciens anno pariunt, quaedam et undeciens, in Aegypto vero etiam brumali mense. hirundines et merulae et palumbi et turtures bis anno pariunt, ceterae aves fere semel. turdi, in cacuminibus arborum luto nidificantes paene contextim, in secessu generant. a coitu X diebus ova maturescunt in utero, vexatis autem gallinae et columbae pinna evulsa aliave simili iniuria diutius.","columbae deciens anno pariunt, quaedam et undeciens, in Aegypto vero etiam brumali mense. hirundines et merulae et palumbi et turtures bis anno pariunt, ceterae aves fere semel. turdi, in cacuminibus arborum luto nidificantes paene contextim, in secessu generant. a coitu X diebus ova maturescunt in utero, vexatis autem gallinae et columbae pinna evulsa aliave simili iniuria diutius."
http://www.zoomathia.com/Pline/10/147,blackbird,"columbae deciens anno pariunt, quaedam et undeciens, in Aegypto vero etiam brumali mense. hirundines et merulae et palumbi et turtures bis anno pariunt, ceterae aves fere semel. turdi, in cacuminibus arborum luto nidificantes paene contextim, in secessu generant. a coitu X diebus ova maturescunt in utero, vexatis autem gallinae et columbae pinna evulsa aliave simili iniuria diutius.","columbae deciens anno pariunt, quaedam et undeciens, in Aegypto vero etiam brumali mense. hirundines et merulae et palumbi et turtures bis anno pariunt, ceterae aves fere semel. turdi, in cacuminibus arborum luto nidificantes paene contextim, in secessu generant. a coitu X diebus ova maturescunt in utero, vexatis autem gallinae et columbae pinna evulsa aliave simili iniuria diutius."
http://www.zoomathia.com/Pline/10/147,turtledove,"columbae deciens anno pariunt, quaedam et undeciens, in Aegypto vero etiam brumali mense. hirundines et merulae et palumbi et turtures bis anno pariunt, ceterae aves fere semel. turdi, in cacuminibus arborum luto nidificantes paene contextim, in secessu generant. a coitu X diebus ova maturescunt in utero, vexatis autem gallinae et columbae pinna evulsa aliave simili iniuria diutius.","columbae deciens anno pariunt, quaedam et undeciens, in Aegypto vero etiam brumali mense. hirundines et merulae et palumbi et turtures bis anno pariunt, ceterae aves fere semel. turdi, in cacuminibus arborum luto nidificantes paene contextim, in secessu generant. a coitu X diebus ova maturescunt in utero, vexatis autem gallinae et columbae pinna evulsa aliave simili iniuria diutius."
http://www.zoomathia.com/Pline/10/147,thrush,"columbae deciens anno pariunt, quaedam et undeciens, in Aegypto vero etiam brumali mense. hirundines et merulae et palumbi et turtures bis anno pariunt, ceterae aves fere semel. turdi, in cacuminibus arborum luto nidificantes paene contextim, in secessu generant. a coitu X diebus ova maturescunt in utero, vexatis autem gallinae et columbae pinna evulsa aliave simili iniuria diutius.","columbae deciens anno pariunt, quaedam et undeciens, in Aegypto vero etiam brumali mense. hirundines et merulae et palumbi et turtures bis anno pariunt, ceterae aves fere semel. turdi, in cacuminibus arborum luto nid

# CQ7

This SPARQL query allows the scientist to target the animals capable of fasting and to understand their
fasting and understand their diets

This query allows to 

In [11]:
SELECT DISTINCT ?paragraph ?name_animal ?mention_animal ?mention_use WHERE {
?annotation1 a oa:Annotation;
              oa:hasBody ?animal;
              oa:hasTarget ?target1.
  ?target1 oa:hasSource ?paragraph;
     oa:hasSelector ?selector.
    
  ?selector oa:exact ?mention_animal.

  ?animal a skos:Concept;
       skos:prefLabel ?name_animal.
    
  ?animal_collection a skos:Collection;
       skos:prefLabel "Ancient class"@en;
       skos:member ?animal.

  ?annotation2 oa:hasBody ?use;
        oa:hasTarget ?target2.
  ?target2 oa:hasSource ?paragraph;
      oa:hasSelector ?selector2.
  ?selector2 oa:exact ?mention_use.

  ?use skos:prefLabel "technical use"@en.

  FILTER (lang(?name_animal) = "en").
}
ORDER BY ?paragraph

paragraph,name_animal,mention_animal,mention_use
http://www.zoomathia.com/Pline/10/2,ostrich,"concoquendi sine dilectu devorata mira natura, sed non minus stoliditas in tanta reliqui corporis altitudine, cum colla frutice occultaverint, latere sese existimantium. praemia ex iis ova, propter amplitudinem pro quibusdam habita vasis, conosque bellicos et galeas adornantes pinnae.","concoquendi sine dilectu devorata mira natura, sed non minus stoliditas in tanta reliqui corporis altitudine, cum colla frutice occultaverint, latere sese existimantium. praemia ex iis ova, propter amplitudinem pro quibusdam habita vasis, conosque bellicos et galeas adornantes pinnae."
http://www.zoomathia.com/Pline/11/103,locust,"cis. in India ternum pedum longitudinis esse traduntur, cruribus et feminibus serrarum usum praebere, cum inaruerint. est et alius earum obitus: gregatim sublatae vento in maria aut stagna decidunt. forte hoc casuque evenit, non, ut prisci existimavere, madefactis nocturno umore alis. iidem quippe nec volare eas noctibus propter frigora tradiderunt, ignari etiam longinqua maria ab iis transiri, continuata plurium dierum_quod maxime miremur_fame","cis. in India ternum pedum longitudinis esse traduntur, cruribus et feminibus serrarum usum praebere, cum inaruerint. est et alius earum obitus: gregatim sublatae vento in maria aut stagna decidunt. forte hoc casuque evenit, non, ut prisci existimavere, madefactis nocturno umore alis. iidem quippe nec volare eas noctibus propter frigora tradiderunt, ignari etiam longinqua maria ab iis transiri, continuata plurium dierum_quod maxime miremur_fame"
http://www.zoomathia.com/Pline/8/124,ass,"Tarandro magnitudo quae bovi est, caput maius cervino nec absimile, cornua ramosa, ungulae bifidae, villus magnitudine ursorum, sed, cum libuit sui coloris esse, asini similis. Tergori tanta duritia, ut thoraces ex eo faciant. colorem omnium arborum, fruticum, florum locorum que reddit metuens in quibus latet, ideoque raro capitur. Mirum esset habitum corpori tam multiplicem dari, mirabilius est et villo.","Tarandro magnitudo quae bovi est, caput maius cervino nec absimile, cornua ramosa, ungulae bifidae, villus magnitudine ursorum, sed, cum libuit sui coloris esse, asini similis. Tergori tanta duritia, ut thoraces ex eo faciant. colorem omnium arborum, fruticum, florum locorum que reddit metuens in quibus latet, ideoque raro capitur. Mirum esset habitum corpori tam multiplicem dari, mirabilius est et villo."
http://www.zoomathia.com/Pline/8/124,deer,"Tarandro magnitudo quae bovi est, caput maius cervino nec absimile, cornua ramosa, ungulae bifidae, villus magnitudine ursorum, sed, cum libuit sui coloris esse, asini similis. Tergori tanta duritia, ut thoraces ex eo faciant. colorem omnium arborum, fruticum, florum locorum que reddit metuens in quibus latet, ideoque raro capitur. Mirum esset habitum corpori tam multiplicem dari, mirabilius est et villo.","Tarandro magnitudo quae bovi est, caput maius cervino nec absimile, cornua ramosa, ungulae bifidae, villus magnitudine ursorum, sed, cum libuit sui coloris esse, asini similis. Tergori tanta duritia, ut thoraces ex eo faciant. colorem omnium arborum, fruticum, florum locorum que reddit metuens in quibus latet, ideoque raro capitur. Mirum esset habitum corpori tam multiplicem dari, mirabilius est et villo."
http://www.zoomathia.com/Pline/8/124,bear,"Tarandro magnitudo quae bovi est, caput maius cervino nec absimile, cornua ramosa, ungulae bifidae, villus magnitudine ursorum, sed, cum libuit sui coloris esse, asini similis. Tergori tanta duritia, ut thoraces ex eo faciant. colorem omnium arborum, fruticum, florum locorum que reddit metuens in quibus latet, ideoque raro capitur. Mirum esset habitum corpori tam multiplicem dari, mirabilius est et villo.","Tarandro magnitudo quae bovi est, caput maius cervino nec absimile, cornua ramosa, ungulae bifidae, villus magnitudine ursorum, sed, cum libuit sui coloris esse, asini similis. Tergori tanta duritia, ut 